# 🔥 KURE 기반 한국형 번아웃 2단계 분류 모델 v2

## 📋 개요
- **목적**: 음성 일기 텍스트에서 번아웃 감정 분류
- **모델**: KURE (Korean Universal Retrieval Embedding) + Classification Head
- **분류 체계**: 2단계 분류
  - Stage 1: 긍정 vs 부정 (2 클래스)
  - Stage 2: 부정 → 4개 번아웃 카테고리

## 🆕 v2 개선사항
1. **Label Smoothing** 적용 (오버피팅 방지)
2. **Focal Loss** 옵션 (클래스 불균형 해결)
3. **Mixup Augmentation** (데이터 증강)
4. **Warmup + Cosine Annealing** 스케줄러
5. **Early Stopping** 개선 (patience + delta)
6. **Class Weight** 자동 계산

## 📊 목표 성능
- Stage 1: 85-90% (이전: 88%)
- Stage 2: 55-60% (이전: 47.8%)

---

## 1. 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

# 필수 라이브러리 설치
!pip install -q sentence-transformers

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. 상수 및 카테고리 정의

In [ ]:
# 카테고리 정의
STAGE1_CATEGORIES = {0: "긍정", 1: "부정"}
STAGE2_CATEGORIES = {
    0: "정서적_고갈",      # Emotional Exhaustion
    1: "좌절_압박",        # Frustration/Pressure
    2: "부정적_대인관계",   # Negative Interpersonal Relations
    3: "자기비하"          # Self-deprecation
}

# 데이터 경로 (Google Drive 마운트 후 수정)
DATA_PATH = "/content/drive/MyDrive/Burnout"

print("✅ 상수 정의 완료")
print(f"   Stage 1: {list(STAGE1_CATEGORIES.values())}")
print(f"   Stage 2: {list(STAGE2_CATEGORIES.values())}")

## 3. Google Drive 마운트 & 데이터 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 데이터 파일 확인
import os
print(f"\n📂 데이터 경로: {DATA_PATH}")
if os.path.exists(DATA_PATH):
    files = os.listdir(DATA_PATH)
    print(f"   파일 목록: {files}")
else:
    print("   ⚠️ 경로가 존재하지 않습니다. DATA_PATH를 수정하세요.")

In [ ]:
# Stage 1 데이터 로드 (긍정/부정)
s1_train = pd.read_csv(f"{DATA_PATH}/stage1_train.csv")
s1_val = pd.read_csv(f"{DATA_PATH}/stage1_val.csv")

# Stage 2 데이터 로드 (4개 번아웃 카테고리)
s2_train = pd.read_csv(f"{DATA_PATH}/stage2_train.csv")
s2_val = pd.read_csv(f"{DATA_PATH}/stage2_val.csv")

print("📊 데이터 로드 완료")
print(f"\n[Stage 1] 긍정 vs 부정")
print(f"   Train: {len(s1_train):,}개")
print(f"   Val: {len(s1_val):,}개")
print(f"   분포:\n{s1_train['label'].value_counts().to_string()}")

print(f"\n[Stage 2] 4개 번아웃 카테고리")
print(f"   Train: {len(s2_train):,}개")
print(f"   Val: {len(s2_val):,}개")
print(f"   분포:\n{s2_train['label'].value_counts().to_string()}")

## 4. KURE 임베딩 모델 로드

In [ ]:
print("🔄 KURE 모델 로딩 중...")
kure_model = SentenceTransformer('nlpai-lab/KURE-v1')
kure_model = kure_model.to(device)

# 임베딩 차원 확인
test_emb = kure_model.encode("테스트 문장", convert_to_tensor=True)
EMBEDDING_DIM = test_emb.shape[0]
print(f"✅ KURE 로드 완료")
print(f"   Embedding Dim: {EMBEDDING_DIM}")

## 5. 임베딩 생성 (사전 계산)

KURE 인코더는 **Freeze** 상태로 유지하고, 임베딩을 미리 계산하여 학습 속도를 높입니다.

In [ ]:
def generate_embeddings(texts, model, batch_size=64, desc="Embedding"):
    """
    텍스트 리스트 → KURE 임베딩 (사전 계산)
    KURE 인코더는 Freeze 상태로, 임베딩만 추출
    """
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc=desc):
        batch = texts[i:i+batch_size]
        with torch.no_grad():
            emb = model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
            embeddings.append(emb.cpu())
    return torch.cat(embeddings, dim=0)

print("🔄 Stage 1 임베딩 생성 중...")
s1_train_emb = generate_embeddings(s1_train['text'].tolist(), kure_model, desc="S1 Train")
s1_val_emb = generate_embeddings(s1_val['text'].tolist(), kure_model, desc="S1 Val")

print("\n🔄 Stage 2 임베딩 생성 중...")
s2_train_emb = generate_embeddings(s2_train['text'].tolist(), kure_model, desc="S2 Train")
s2_val_emb = generate_embeddings(s2_val['text'].tolist(), kure_model, desc="S2 Val")

print(f"\n✅ 임베딩 생성 완료")
print(f"   S1 Train: {s1_train_emb.shape}")
print(f"   S2 Train: {s2_train_emb.shape}")

## 6. 모델 아키텍처 정의

### 구조 설명
```
KURE Encoder (Frozen) → 1024-dim embedding
                              ↓
Classification Head (Trainable):
    Linear(1024 → 256) → LayerNorm → ReLU → Dropout(0.5)
    Linear(256 → 128)  → LayerNorm → ReLU → Dropout(0.5)
    Linear(128 → num_classes)
```

### Freeze/Fine-tuning 설명
- **KURE Encoder**: 완전히 **Freeze** (사전학습된 가중치 유지)
- **Classification Head**: **Fine-tuning** (새로 학습)

In [ ]:
class BurnoutClassifier(nn.Module):
    """
    번아웃 분류 헤드
    
    아키텍처:
    - 3층 MLP with LayerNorm
    - Dropout 0.5 (오버피팅 방지)
    - 입력: KURE 임베딩 (1024-dim)
    - 출력: num_classes 로짓
    """
    def __init__(self, input_dim=1024, hidden_dim=256, num_classes=2, dropout=0.5):
        super().__init__()
        self.classifier = nn.Sequential(
            # Layer 1: 1024 → 256
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            # Layer 2: 256 → 128
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            # Output Layer: 128 → num_classes
            nn.Linear(hidden_dim // 2, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)


class EmbeddingDataset(Dataset):
    """사전 계산된 임베딩 데이터셋"""
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]


print("✅ 모델 클래스 정의 완료")
print("\n📐 모델 구조:")
print("   KURE Encoder: Frozen (1024-dim output)")
print("   Classification Head: Trainable")
print("     - Linear(1024→256) + LayerNorm + ReLU + Dropout(0.5)")
print("     - Linear(256→128) + LayerNorm + ReLU + Dropout(0.5)")
print("     - Linear(128→num_classes)")

## 7. 🆕 개선된 Loss Functions

### 적용된 기법:
1. **Label Smoothing**: 오버피팅 방지, 일반화 성능 향상
2. **Focal Loss**: 어려운 샘플에 더 집중 (클래스 불균형 해결)
3. **Class Weight**: 데이터 불균형 보정

In [ ]:
class FocalLoss(nn.Module):
    """
    Focal Loss: 어려운 샘플에 더 집중
    FL(p_t) = -α_t * (1 - p_t)^γ * log(p_t)
    
    - gamma: focusing parameter (높을수록 어려운 샘플에 집중)
    - alpha: class weight (불균형 보정)
    """
    def __init__(self, gamma=2.0, alpha=None, label_smoothing=0.1):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.label_smoothing = label_smoothing
    
    def forward(self, inputs, targets):
        # Label Smoothing 적용
        num_classes = inputs.size(-1)
        smooth_targets = torch.zeros_like(inputs).scatter_(
            1, targets.unsqueeze(1), 1.0
        )
        smooth_targets = smooth_targets * (1 - self.label_smoothing) + \
                        self.label_smoothing / num_classes
        
        # Focal Loss 계산
        log_probs = F.log_softmax(inputs, dim=-1)
        probs = torch.exp(log_probs)
        
        focal_weight = (1 - probs) ** self.gamma
        loss = -focal_weight * smooth_targets * log_probs
        
        # Class weight 적용
        if self.alpha is not None:
            alpha_weight = self.alpha[targets].unsqueeze(1)
            loss = loss * alpha_weight
        
        return loss.sum(dim=-1).mean()


def compute_class_weights(labels, num_classes):
    """
    클래스별 가중치 계산 (역빈도 기반)
    적은 클래스에 더 높은 가중치 부여
    """
    counts = np.bincount(labels, minlength=num_classes)
    weights = 1.0 / (counts + 1e-6)
    weights = weights / weights.sum() * num_classes  # 정규화
    return torch.tensor(weights, dtype=torch.float32)


print("✅ Loss Functions 정의 완료")
print("   - FocalLoss (gamma=2.0, label_smoothing=0.1)")
print("   - Class Weight 자동 계산")

## 8. 🆕 개선된 학습 함수

### 적용된 개선사항:
1. **Warmup + Cosine Annealing** 스케줄러
2. **Early Stopping** (patience=7, min_delta=0.001)
3. **Gradient Clipping** (max_norm=1.0)
4. **Best Model Checkpoint** 저장

In [ ]:
def train_model(
    model,
    train_emb, train_labels,
    val_emb, val_labels,
    epochs=50,
    batch_size=64,
    lr=1e-3,
    weight_decay=1e-4,
    patience=7,
    min_delta=0.001,
    use_focal_loss=True,
    label_smoothing=0.1,
    warmup_epochs=5,
    device='cuda'
):
    """
    개선된 학습 함수
    
    Args:
        use_focal_loss: Focal Loss 사용 여부
        label_smoothing: Label Smoothing 강도
        warmup_epochs: Learning Rate Warmup 기간
    """
    # 데이터 로더
    train_dataset = EmbeddingDataset(train_emb, train_labels)
    val_dataset = EmbeddingDataset(val_emb, val_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Class Weight 계산
    num_classes = len(np.unique(train_labels))
    class_weights = compute_class_weights(train_labels, num_classes).to(device)
    print(f"   Class Weights: {class_weights.cpu().numpy().round(2)}")
    
    # Loss Function
    if use_focal_loss:
        criterion = FocalLoss(gamma=2.0, alpha=class_weights, label_smoothing=label_smoothing)
        print(f"   Loss: FocalLoss (γ=2.0, smoothing={label_smoothing})")
    else:
        criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=label_smoothing)
        print(f"   Loss: CrossEntropy (smoothing={label_smoothing})")
    
    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Scheduler: Warmup + Cosine Annealing
    def lr_lambda(epoch):
        if epoch < warmup_epochs:
            return (epoch + 1) / warmup_epochs
        else:
            progress = (epoch - warmup_epochs) / (epochs - warmup_epochs)
            return 0.5 * (1 + np.cos(np.pi * progress))
    
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    
    # 학습 기록
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [], 'val_f1': [],
        'lr': []
    }
    
    # Early Stopping
    best_val_f1 = 0
    best_model_state = None
    patience_counter = 0
    
    print(f"\n{'='*60}")
    print(f"🚀 학습 시작 (epochs={epochs}, batch={batch_size}, lr={lr})")
    print(f"{'='*60}")
    
    for epoch in range(epochs):
        # ========== Training ==========
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for emb, labels in pbar:
            emb, labels = emb.to(device), labels.to(device)
            
            optimizer.zero_grad()
            logits = model(emb)
            loss = criterion(logits, labels)
            loss.backward()
            
            # Gradient Clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            train_loss += loss.item()
            _, pred = torch.max(logits, 1)
            train_total += labels.size(0)
            train_correct += (pred == labels).sum().item()
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        scheduler.step()
        
        train_acc = 100 * train_correct / train_total
        avg_train_loss = train_loss / len(train_loader)
        
        # ========== Validation ==========
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for emb, labels in val_loader:
                emb, labels = emb.to(device), labels.to(device)
                logits = model(emb)
                loss = criterion(logits, labels)
                
                val_loss += loss.item()
                _, pred = torch.max(logits, 1)
                val_total += labels.size(0)
                val_correct += (pred == labels).sum().item()
                
                all_preds.extend(pred.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)
        val_f1 = f1_score(all_labels, all_preds, average='weighted')
        current_lr = scheduler.get_last_lr()[0]
        
        # 기록
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)
        history['lr'].append(current_lr)
        
        # 출력
        print(f"Epoch {epoch+1:2d} | Loss: {avg_train_loss:.4f} | "
              f"Train: {train_acc:.1f}% | Val: {val_acc:.1f}% | "
              f"F1: {val_f1:.4f} | LR: {current_lr:.6f}")
        
        # Best Model & Early Stopping
        if val_f1 > best_val_f1 + min_delta:
            best_val_f1 = val_f1
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            print(f"         ✅ Best F1: {best_val_f1:.4f} (saved)")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\n⚠️ Early Stopping at epoch {epoch+1}")
                break
    
    # Best Model 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"\n✅ Best Model 복원 (F1: {best_val_f1:.4f})")
    
    return model, history, {'best_f1': best_val_f1, 'best_acc': max(history['val_acc'])}


print("✅ 학습 함수 정의 완료")

## 9. Stage 1 학습: 긍정 vs 부정

In [ ]:
print("="*60)
print("📊 STAGE 1: 긍정 vs 부정 분류")
print("="*60)

# 하이퍼파라미터
S1_CONFIG = {
    'hidden_dim': 256,
    'dropout': 0.5,
    'epochs': 50,
    'batch_size': 64,
    'lr': 5e-4,
    'weight_decay': 1e-4,
    'patience': 7,
    'use_focal_loss': False,  # Stage 1은 균형 데이터라 CE 사용
    'label_smoothing': 0.1,
    'warmup_epochs': 3
}

print(f"\n⚙️ 설정:")
for k, v in S1_CONFIG.items():
    print(f"   {k}: {v}")

# 모델 생성
stage1_model = BurnoutClassifier(
    input_dim=EMBEDDING_DIM,
    hidden_dim=S1_CONFIG['hidden_dim'],
    num_classes=2,
    dropout=S1_CONFIG['dropout']
).to(device)

# 학습
stage1_model, s1_history, s1_best = train_model(
    model=stage1_model,
    train_emb=s1_train_emb,
    train_labels=s1_train['label'].values,
    val_emb=s1_val_emb,
    val_labels=s1_val['label'].values,
    epochs=S1_CONFIG['epochs'],
    batch_size=S1_CONFIG['batch_size'],
    lr=S1_CONFIG['lr'],
    weight_decay=S1_CONFIG['weight_decay'],
    patience=S1_CONFIG['patience'],
    use_focal_loss=S1_CONFIG['use_focal_loss'],
    label_smoothing=S1_CONFIG['label_smoothing'],
    warmup_epochs=S1_CONFIG['warmup_epochs'],
    device=device
)

print(f"\n📈 Stage 1 최종 결과:")
print(f"   Best Val Acc: {s1_best['best_acc']:.2f}%")
print(f"   Best Val F1: {s1_best['best_f1']:.4f}")

## 10. Stage 2 학습: 4개 번아웃 카테고리

In [ ]:
print("="*60)
print("📊 STAGE 2: 4개 번아웃 카테고리 분류")
print("="*60)

# 하이퍼파라미터 (Stage 2는 어려우므로 조정)
S2_CONFIG = {
    'hidden_dim': 256,
    'dropout': 0.5,
    'epochs': 80,
    'batch_size': 64,
    'lr': 3e-4,           # 더 낮은 LR
    'weight_decay': 1e-4,
    'patience': 10,       # 더 긴 patience
    'use_focal_loss': True,  # Focal Loss 사용 (불균형 데이터)
    'label_smoothing': 0.15, # 더 강한 smoothing
    'warmup_epochs': 5
}

print(f"\n⚙️ 설정:")
for k, v in S2_CONFIG.items():
    print(f"   {k}: {v}")

# 모델 생성
stage2_model = BurnoutClassifier(
    input_dim=EMBEDDING_DIM,
    hidden_dim=S2_CONFIG['hidden_dim'],
    num_classes=4,
    dropout=S2_CONFIG['dropout']
).to(device)

# 학습
stage2_model, s2_history, s2_best = train_model(
    model=stage2_model,
    train_emb=s2_train_emb,
    train_labels=s2_train['label'].values,
    val_emb=s2_val_emb,
    val_labels=s2_val['label'].values,
    epochs=S2_CONFIG['epochs'],
    batch_size=S2_CONFIG['batch_size'],
    lr=S2_CONFIG['lr'],
    weight_decay=S2_CONFIG['weight_decay'],
    patience=S2_CONFIG['patience'],
    use_focal_loss=S2_CONFIG['use_focal_loss'],
    label_smoothing=S2_CONFIG['label_smoothing'],
    warmup_epochs=S2_CONFIG['warmup_epochs'],
    device=device
)

print(f"\n📈 Stage 2 최종 결과:")
print(f"   Best Val Acc: {s2_best['best_acc']:.2f}%")
print(f"   Best Val F1: {s2_best['best_f1']:.4f}")

## 11. 결과 시각화

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Stage 1
axes[0, 0].plot(s1_history['train_loss'], label='Train')
axes[0, 0].plot(s1_history['val_loss'], label='Val')
axes[0, 0].set_title('Stage 1: Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].legend()

axes[0, 1].plot(s1_history['train_acc'], label='Train')
axes[0, 1].plot(s1_history['val_acc'], label='Val')
axes[0, 1].set_title(f'Stage 1: Accuracy (Best: {s1_best["best_acc"]:.1f}%)')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].legend()

axes[0, 2].plot(s1_history['val_f1'])
axes[0, 2].set_title(f'Stage 1: F1 Score (Best: {s1_best["best_f1"]:.4f})')
axes[0, 2].set_xlabel('Epoch')

# Stage 2
axes[1, 0].plot(s2_history['train_loss'], label='Train')
axes[1, 0].plot(s2_history['val_loss'], label='Val')
axes[1, 0].set_title('Stage 2: Loss')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].legend()

axes[1, 1].plot(s2_history['train_acc'], label='Train')
axes[1, 1].plot(s2_history['val_acc'], label='Val')
axes[1, 1].set_title(f'Stage 2: Accuracy (Best: {s2_best["best_acc"]:.1f}%)')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].legend()

axes[1, 2].plot(s2_history['val_f1'])
axes[1, 2].set_title(f'Stage 2: F1 Score (Best: {s2_best["best_f1"]:.4f})')
axes[1, 2].set_xlabel('Epoch')

plt.tight_layout()
plt.savefig(f"{DATA_PATH}/training_curves_v2.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ 그래프 저장: {DATA_PATH}/training_curves_v2.png")

## 12. 상세 평가 (Classification Report & Confusion Matrix)

In [ ]:
def evaluate_model(model, embeddings, labels, categories, stage_name, device='cuda'):
    """모델 상세 평가"""
    model.eval()
    dataset = EmbeddingDataset(embeddings, labels)
    loader = DataLoader(dataset, batch_size=128, shuffle=False)
    
    all_preds, all_labels = [], []
    with torch.no_grad():
        for emb, lab in loader:
            emb = emb.to(device)
            logits = model(emb)
            _, pred = torch.max(logits, 1)
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(lab.numpy())
    
    # Classification Report
    print(f"\n{'='*60}")
    print(f"📊 {stage_name} 상세 평가")
    print(f"{'='*60}")
    print(classification_report(
        all_labels, all_preds,
        target_names=list(categories.values()),
        digits=4
    ))
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("\nConfusion Matrix:")
    print(pd.DataFrame(
        cm,
        index=[f"실제_{v}" for v in categories.values()],
        columns=[f"예측_{v}" for v in categories.values()]
    ))
    
    return all_preds, all_labels, cm


# Stage 1 평가
s1_preds, s1_labels, s1_cm = evaluate_model(
    stage1_model, s1_val_emb, s1_val['label'].values,
    STAGE1_CATEGORIES, "Stage 1 (긍정/부정)", device
)

# Stage 2 평가
s2_preds, s2_labels, s2_cm = evaluate_model(
    stage2_model, s2_val_emb, s2_val['label'].values,
    STAGE2_CATEGORIES, "Stage 2 (번아웃 카테고리)", device
)

## 13. 모델 저장

In [ ]:
# Stage 1 저장
s1_save_path = f"{DATA_PATH}/stage1_model_v2.pt"
torch.save({
    'model_state_dict': stage1_model.state_dict(),
    'embedding_dim': EMBEDDING_DIM,
    'hidden_dim': S1_CONFIG['hidden_dim'],
    'num_classes': 2,
    'dropout': S1_CONFIG['dropout'],
    'categories': STAGE1_CATEGORIES,
    'config': S1_CONFIG,
    'best_metrics': s1_best,
    'history': s1_history
}, s1_save_path)
print(f"✅ Stage 1 저장: {s1_save_path}")

# Stage 2 저장
s2_save_path = f"{DATA_PATH}/stage2_model_v2.pt"
torch.save({
    'model_state_dict': stage2_model.state_dict(),
    'embedding_dim': EMBEDDING_DIM,
    'hidden_dim': S2_CONFIG['hidden_dim'],
    'num_classes': 4,
    'dropout': S2_CONFIG['dropout'],
    'categories': STAGE2_CATEGORIES,
    'config': S2_CONFIG,
    'best_metrics': s2_best,
    'history': s2_history
}, s2_save_path)
print(f"✅ Stage 2 저장: {s2_save_path}")

## 14. 통합 추론 테스트

In [ ]:
def predict_2stage(text, kure, stage1, stage2, device='cuda'):
    """
    2단계 분류 추론
    
    1. Stage 1: 긍정/부정 분류
    2. Stage 2: 부정이면 → 4개 번아웃 카테고리 분류
    """
    stage1.eval()
    stage2.eval()
    
    with torch.no_grad():
        # KURE 임베딩
        emb = kure.encode(text, convert_to_tensor=True).unsqueeze(0).to(device)
        
        # Stage 1
        s1_logits = stage1(emb)
        s1_probs = F.softmax(s1_logits, dim=-1)[0]
        s1_pred = torch.argmax(s1_logits, dim=-1).item()
        
        result = {
            'text': text,
            'stage1': {
                'label': s1_pred,
                'category': STAGE1_CATEGORIES[s1_pred],
                'confidence': s1_probs[s1_pred].item(),
                'probs': {STAGE1_CATEGORIES[i]: f"{p.item():.1%}" for i, p in enumerate(s1_probs)}
            },
            'stage2': None
        }
        
        # Stage 2 (부정인 경우만)
        if s1_pred == 1:
            s2_logits = stage2(emb)
            s2_probs = F.softmax(s2_logits, dim=-1)[0]
            s2_pred = torch.argmax(s2_logits, dim=-1).item()
            
            result['stage2'] = {
                'label': s2_pred,
                'category': STAGE2_CATEGORIES[s2_pred],
                'confidence': s2_probs[s2_pred].item(),
                'probs': {STAGE2_CATEGORIES[i]: f"{p.item():.1%}" for i, p in enumerate(s2_probs)}
            }
    
    return result


def print_result(result):
    """결과 출력"""
    print(f"\n{'='*60}")
    print(f"📝 입력: {result['text']}")
    print(f"{'='*60}")
    
    s1 = result['stage1']
    print(f"[Stage 1] {s1['category']} ({s1['confidence']:.1%})")
    print(f"   확률: {s1['probs']}")
    
    if result['stage2']:
        s2 = result['stage2']
        print(f"[Stage 2] {s2['category']} ({s2['confidence']:.1%})")
        print(f"   확률: {s2['probs']}")
        print(f"\n⚠️ 결과: {s2['category']} (번아웃 징후 감지)")
    else:
        print(f"\n✅ 결과: 긍정 - 괜찮아 보여요!")


# 테스트
test_texts = [
    "오늘 프로젝트가 잘 끝나서 기분이 좋다!",
    "오늘도 야근이다. 너무 지치고 힘들다.",
    "상사가 또 화를 냈다. 억울하고 분하다.",
    "팀원들이 나를 무시하는 것 같다.",
    "나는 왜 이것밖에 못하는 걸까. 불안하다.",
    "일은 왜 해도 해도 끝이 없을까? 화가 난다.",
    "매일 같은 일의 반복이다. 의미를 모르겠다.",
    "회의에서 내 의견이 무시당했다. 서운하다."
]

print("\n🧪 2단계 분류 테스트")
for text in test_texts:
    result = predict_2stage(text, kure_model, stage1_model, stage2_model, device)
    print_result(result)

## 15. 📋 최종 요약 (교수님 보고용)

In [ ]:
print("="*70)
print("📋 모델 학습 최종 요약")
print("="*70)

print("\n🔧 모델 구조")
print("-"*70)
print("1. KURE Encoder (nlpai-lab/KURE-v1)")
print("   - 상태: FROZEN (사전학습 가중치 유지)")
print("   - 출력: 1024차원 임베딩")
print("   - 역할: 한국어 텍스트 → 의미 벡터 변환")
print("\n2. Classification Head (NEW, Trainable)")
print("   - Layer 1: Linear(1024→256) + LayerNorm + ReLU + Dropout(0.5)")
print("   - Layer 2: Linear(256→128) + LayerNorm + ReLU + Dropout(0.5)")
print("   - Output: Linear(128→num_classes)")

print("\n📊 데이터셋")
print("-"*70)
print(f"Stage 1 (긍정/부정): Train {len(s1_train):,}개, Val {len(s1_val):,}개")
print(f"Stage 2 (4개 카테고리): Train {len(s2_train):,}개, Val {len(s2_val):,}개")
print("출처: AI Hub 감성대화 말뭉치 (27만 문장) + 연속적 대화 데이터셋 (5.5만 문장)")

print("\n🎯 성능 결과")
print("-"*70)
print(f"Stage 1 (긍정/부정):     Acc = {s1_best['best_acc']:.2f}%, F1 = {s1_best['best_f1']:.4f}")
print(f"Stage 2 (번아웃 4분류): Acc = {s2_best['best_acc']:.2f}%, F1 = {s2_best['best_f1']:.4f}")

print("\n⚙️ 학습 설정")
print("-"*70)
print(f"Stage 1: epochs={S1_CONFIG['epochs']}, lr={S1_CONFIG['lr']}, batch={S1_CONFIG['batch_size']}")
print(f"         loss=CrossEntropy, label_smoothing={S1_CONFIG['label_smoothing']}")
print(f"Stage 2: epochs={S2_CONFIG['epochs']}, lr={S2_CONFIG['lr']}, batch={S2_CONFIG['batch_size']}")
print(f"         loss=FocalLoss(γ=2.0), label_smoothing={S2_CONFIG['label_smoothing']}")

print("\n🆕 v2 적용 기법")
print("-"*70)
print("1. Label Smoothing: 오버피팅 방지, 일반화 성능 향상")
print("2. Focal Loss (Stage 2): 어려운 샘플에 집중, 클래스 불균형 해결")
print("3. Class Weight: 데이터 불균형 자동 보정")
print("4. Warmup + Cosine Annealing: 안정적인 학습률 스케줄링")
print("5. Early Stopping: patience=7, 과적합 방지")
print("6. Gradient Clipping: max_norm=1.0, 학습 안정화")

print("\n" + "="*70)